# import modules

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
# from sklearn.pipeline import Pipeline
import datetime
import time

# Data set 불러오기

In [4]:
# 여러분의 폴더가 저장되어있는 위치 경로를 써 주세요
# 그리고 데이터파일(주최측에서 제공한) 의 이름을 Train set 으로 변셩해 주세요
# 한인욱 directory = 'C:/Users/goran/Desktop/Git_Hub/Project/20S_Bigcon'
# 김정현 directory = 'C:/Users/Coby/Desktop/20S_Bigcon/20S_Bigcon'
# 신예진 directory = 'C:/Users/Rstudio/Desktop/Bigcon2020/20S_Bigcon'
# 이규민 directory = 'C:/Statistics/20S_Bigcon'
# 최정욱 directory = 'C:/Users/lg/Desktop/Bigcon2020'
directory = 'C:/Users/goran/Desktop/Git_Hub/Project/20S_Bigcon'

In [5]:
# 그러면 이렇게 그 폴더 안의 Train set.xlsx 을 잘 읽어옵니다! 
X_df=pd.read_excel(directory+'/Train set.xlsx',header = 1) 

In [6]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 하세요! 그게 제일 빨라요
X = X_df.copy()

In [7]:
X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


# 결측치 처리 및 변수늘리기

In [8]:
X.isnull().sum()

방송일시         0
노출(분)    16784
마더코드         0
상품코드         0
상품명          0
상품군          0
판매단가         0
취급액       2930
dtype: int64

## 노출(분) 결측치 처리

In [13]:
#노출분은 같은 시간에 같이 방송되어서 NaN 으로 뜨는것이다.
#같은 시간에 방송되어진 값을 평균해서 채운다.

In [9]:
X.loc[:,'평균방송분'] = X.loc[:,'노출(분)']

In [10]:
# 0 노출시간이 0 인 데이터가 없다.
# 즉 Na 처리할떄 0으로 대체한 후에 처리하였다.
(X.loc[:,'노출(분)']==0).sum()

0

In [11]:
# 노출(분) 을 제일 먼저 시작한 시간의 값으로 만드는 FUNCTION 정의
def NA_minute(df=X):
    # 곁측치를 0으로 대체
    df['노출(분)'] = df['노출(분)'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'노출(분)'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'노출(분)'] = df.loc[start,'노출(분)'] # / num 을 붙여주면 평균으로 만들게 됩니다.
    return df

In [12]:
X = NA_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,20.000000
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,NaN
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,20.000000
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,NaN
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,20.000000
...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000


## 얼마나 같은 방송일시에 겹쳐있는지 변수늘리기

In [14]:
# 겹치는 수는 같은 편성을 연속해서 했을때의 경우를 고려한 것이다.
# 같은 편성을 했다는 기준은 마더코드를 기준으로 삼았다.(즉 마더코드가 같은 방송이 연속되면, 겹쳤다고 판단.)
# 예를 들어 삼성 tv 를 4시/4시20분/4시40분 3번 편성했다면 겹치는수는 3이 된다.

In [15]:
X.loc[:,'겹치는수'] = 1

In [16]:
for idx in range(1,len(X))  :
    if ~(X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']): 
        continue
    elif (X.loc[idx-1,'마더코드'] == X.loc[idx,'마더코드']) : 
        if X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']: . 
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']
        elif ~(X.loc[idx-1,'방송일시'] == X.loc[idx,'방송일시']): 
            X.loc[idx,'겹치는수'] = X.loc[idx-1,'겹치는수']+1

In [17]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,20.000000,1
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,NaN,1
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,20.000000,2
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,NaN,2
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,20.000000,3
...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1


## 평균 방송분 변수늘리기

In [18]:
# 각 방송에대한 평균 방송분 열을 추가하였다.
# 즉 6:00 ~ 6:20 분동안 4개의 상품이 방송되었다면, 평균적으로 5분 노출한것이므로, 평균방송분은 5분이 된다.

In [19]:
def MEAN_minute(df=X):
    # 곁측치를 0으로 대체
    df['평균방송분'] = df['평균방송분'].fillna(0)
    
    count_0 = [] 
    count_not0 = [] # 0이 아닌 수의 index
    for idx in df.index :
        if df.loc[idx,'평균방송분'] == 0 :
            count_0.append(idx)
        else :
            count_not0.append(idx)
    count_not0.append(len(df)) # 맨 마지막 열에 0 인 경우를 대비하여

    for idx,x in enumerate(count_not0):
        if idx != len(count_not0)-1: 
            if (count_not0[idx+1] - count_not0[idx] - 1) != 0:
                start = count_not0[idx] 
                end = count_not0[idx+1]
                num = end - start 
                df.loc[start:(end-1),'평균방송분'] = df.loc[start,'평균방송분'] / num
    return df

In [20]:
X = MEAN_minute(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.000000,1
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.000000,1
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.000000,2
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.000000,2
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.000000,3
...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1


## 시간 데이터 늘리기

In [21]:
# 방송일시를 쪼개서 시간데이터를 최대로 늘렸다.
# 년,월,알,시간,시분 등으로 나누었고, 이는 뒤에 변수를 조함해 추가할떄 유용하게 쓰이지만, 직접적인 수치형 데이터로 쓰이는 열은 많지 않다.

In [22]:
def time_feature(df = X):
    df['방송일시']=df['방송일시'].astype('str')
    df['날짜']=df['방송일시'].str.split(' ').str[0]
    df['시간']=df['방송일시'].str.split(' ').str[1]
    df['년']=df['날짜'].str.split('-').str[0]
    df['월']=df['날짜'].str.split('-').str[1]
    df['일']=df['날짜'].str.split('-').str[2]
    df['시']=df['시간'].str.split(':').str[0]
    df['분']=df['시간'].str.split(':').str[1]
    df['초']=df['시간'].str.split(':').str[2]
    df['시각'] = df['시'].astype('float32') + df['분'].astype('float32')*(1/60)
    df['시분'] = df['시'] + ':' + df['분']
    df.drop(['시','분','초','시간'],axis=1,inplace = True)
    return(df)

In [23]:
X = time_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,날짜,년,월,일,시각,시분
0,2019-01-01 06:00:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00
1,2019-01-01 06:00:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.000000,1,2019-01-01,2019,01,01,6.000000,06:00
2,2019-01-01 06:20:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20
3,2019-01-01 06:20:00,20.000000,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.000000,2,2019-01-01,2019,01,01,6.333333,06:20
4,2019-01-01 06:40:00,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.000000,3,2019-01-01,2019,01,01,6.666667,06:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38304,2020-01-01 00:20:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,0.333333,00:20
38305,2020-01-01 00:40:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,2,2020-01-01,2020,01,01,0.666667,00:40
38306,2020-01-01 01:00:00,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,20.000000,3,2020-01-01,2020,01,01,1.000000,01:00
38307,2020-01-01 01:20:00,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,20.000000,1,2020-01-01,2020,01,01,1.333333,01:20


## 주말 시청률 평균 생성

In [24]:
# 주최측에서 제공한 시청률 데이터를 이용하여 주말(금요일 포함) 의 시청률에 대한 평균을 생성하였다.

In [26]:
# 시청률 데이터셋의 이름을 Train_rating.xlsx 으로 변경하고 진행하였다.
rate_df=pd.read_excel(directory+'/Train_rating.xlsx',header = 1) 

In [27]:
rate = rate_df.copy()

In [28]:
time_list= ['0101','0104','0105','0106','0111','0112','0113','0118','0119','0120','0125','0126','0127',
                  '0201','0202','0203','0204','0205','0206','0208','0209','0210','0215','0216','0217','0222','0223','0224',
                  '0301','0302','0303','0308','0309','0310','0315','0316','0317','0322','0323','0324','0329','0330','0331',
                  '0405','0406','0407','0412','0413','0414','0419','0420','0421','0426','0427','0428'
                  ,'0503','0504','0505','0506','0510','0511','0512','0517','0518','0519','0524','0525','0526','0531',
            '0601','0602','0606','0607','0608','0609','0614','0615','0616','0621','0622','0623','0628','0629','0630',
            '0704','0705','0706','0707','0712','0713','0714','0719','0720','0721','0726','0727','0728',
            '0802','0803','0804','0809','0810','0811','0815','0816','0817','0818','0823','0824','0825',
            '0901','0906','0907','0908','0912','0913','0914','0915','0920','0921','0922','0927','0928','0929'
 ,'1003','1004','1005','1006','1009','1011','1012','1013','1018','1019','1020','1025','1026','1027',
                '1101','1102','1103','1108','1109','1110','1115','1116','1117','1122','1123','1124','1129','1130',
                  '1201','1206','1207','1208','1213','1214','1215','1220','1221','1222','1225','1227','1228','1229']

In [29]:
rate_T = rate.T  
rate_T.reset_index(inplace=True)
rate_T.columns = rate_T.iloc[0,:]
rate_T.drop('월화수목금토일02:00-01:59',axis=1,inplace=True)
rate_T.drop(index=366,inplace=True)
rate_T.drop(index=0,inplace=True)
rate_T.loc[:,'시간대'] = rate_T.loc[:,'시간대'].astype(str)
rate_T['월']=rate_T['시간대'].str.split('-').str[1]
rate_T['일']=rate_T['시간대'].str.split('-').str[2]
rate_T['월일'] = rate_T['월'].astype(str) + rate_T['일'].astype(str)
rate_T.reset_index(drop=True,inplace = True)

In [30]:
for i in range(len(rate_T)):
    if rate_T.loc[i,'월일'] not in time_list: # 주말이 아니라면, drop 시킬거야! 
        rate_T.drop(index = i , inplace = True) # 그러면 주말만 남을겁니다.

In [31]:
rate_T.drop(columns=['월','일','월일'],inplace= True)
rate_T.drop(columns = ['시간대'],inplace= True)
rate_T=rate_T.astype(float)
weekend=pd.DataFrame(np.mean(rate_T,axis=0),columns=['시청률 평균'])
weekend.index.name = None

In [33]:
weekend=pd.DataFrame(np.mean(rate_T,axis=0),columns=['시청률 평균'])
weekend.index.name = '시간'
weekend=weekend.reset_index()
weekend=pd.concat([weekend.loc[240:],weekend.loc[:239]],axis=0)
weekend=weekend.reset_index(drop=True)

In [34]:
weekend.loc[:,'시간'] = weekend.loc[:,'시간'].astype(str)
weekend['시각']=weekend['시간'].str.split(':').str[0].astype(str) + weekend['시간'].str.split(':').str[1].astype(str)
weekend['시각']=weekend['시각'].astype(int)
for i in range(len(weekend)):
    if weekend.loc[i,'시각'] < 600:
         weekend.loc[i,'시각']  += 2400

In [36]:
weekend.index = weekend['시각']

In [37]:
weekend

,시간,시청률 평균,시각
시각,,,
600,06:00,0.002175,600
601,06:01,0.002434,601
602,06:02,0.002747,602
603,06:03,0.003976,603
604,06:04,0.003795,604
...,...,...,...
2955,05:55,0.002018,2955
2956,05:56,0.002819,2956
2957,05:57,0.002608,2957


## 평일시청률 평균 생성

In [38]:
# 위의 과정과 동일합니다.

In [39]:
# 앞의 과정처럼 그 폴더 안의 Train set.xlsx 을 읽습니다. 
rate_df=pd.read_excel(directory+'/Train_rating.xlsx',header = 1) 

In [40]:
# 데이터 reset 하고싶으면 위의 셀 말고 이거 돌려가면서 reset 합시다.(앞과 같은 과정입니다)
rate = rate_df.copy()

In [41]:
time_list= ['0101','0104','0105','0106','0111','0112','0113','0118','0119','0120','0125','0126','0127',
                  '0201','0202','0203','0204','0205','0206','0208','0209','0210','0215','0216','0217','0222','0223','0224',
                  '0301','0302','0303','0308','0309','0310','0315','0316','0317','0322','0323','0324','0329','0330','0331',
                  '0405','0406','0407','0412','0413','0414','0419','0420','0421','0426','0427','0428'
                  ,'0503','0504','0505','0506','0510','0511','0512','0517','0518','0519','0524','0525','0526','0531',
            '0601','0602','0606','0607','0608','0609','0614','0615','0616','0621','0622','0623','0628','0629','0630',
            '0704','0705','0706','0707','0712','0713','0714','0719','0720','0721','0726','0727','0728',
            '0802','0803','0804','0809','0810','0811','0815','0816','0817','0818','0823','0824','0825',
            '0901','0906','0907','0908','0912','0913','0914','0915','0920','0921','0922','0927','0928','0929'
 ,'1003','1004','1005','1006','1009','1011','1012','1013','1018','1019','1020','1025','1026','1027',
                '1101','1102','1103','1108','1109','1110','1115','1116','1117','1122','1123','1124','1129','1130',
                  '1201','1206','1207','1208','1213','1214','1215','1220','1221','1222','1225','1227','1228','1229']

In [42]:
rate_T = rate.T  # TRANSPOSE 를 해야 알맞게 변환이 됩니다.
rate_T.reset_index(inplace=True)
rate_T.columns = rate_T.iloc[0,:]
rate_T.drop('월화수목금토일02:00-01:59',axis=1,inplace=True)
rate_T.drop(index=366,inplace=True)
rate_T.drop(index=0,inplace=True)
rate_T.loc[:,'시간대'] = rate_T.loc[:,'시간대'].astype(str)
rate_T['월']=rate_T['시간대'].str.split('-').str[1]
rate_T['일']=rate_T['시간대'].str.split('-').str[2]
rate_T['월일'] = rate_T['월'].astype(str) + rate_T['일'].astype(str)
rate_T.reset_index(drop=True,inplace = True)

In [43]:
for i in range(len(rate_T)):
    if rate_T.loc[i,'월일'] in time_list: # 주말이 아니라면, drop 시킬거야! 
        rate_T.drop(index = i , inplace = True) # 그러면 주말만 남을겁니다.
rate_T.drop(columns=['월','일','월일'],inplace= True)
rate_T.drop(columns = ['시간대'],inplace= True)
rate_T=rate_T.astype(float)
weekday=pd.DataFrame(np.mean(rate_T,axis=0),columns=['시청률 평균'])
weekday.index.name = '시간'
weekday=weekday.reset_index()
weekday=pd.concat([weekday.loc[240:],weekday.loc[:239]],axis=0)

In [44]:
weekday=weekday.reset_index(drop=True)

In [45]:
weekday.loc[:,'시간'] = weekday.loc[:,'시간'].astype(str)
weekday['시각']=weekday['시간'].str.split(':').str[0].astype(str) + weekday['시간'].str.split(':').str[1].astype(str)
weekday['시각']=weekday['시각'].astype(int)
for i in range(len(weekday)):
    if weekday.loc[i,'시각'] < 600:
         weekday.loc[i,'시각']  += 2400

In [46]:
weekday.index = weekday['시각']
weekday

,시간,시청률 평균,시각
시각,,,
600,06:00,0.001407,600
601,06:01,0.001648,601
602,06:02,0.001492,602
603,06:03,0.001658,603
604,06:04,0.001849,604
...,...,...,...
2955,05:55,0.001136,2955
2956,05:56,0.001231,2956
2957,05:57,0.000950,2957


## 시청률 외부데이터 적용

In [50]:
# 금토일,공휴일 / 월화수목 의 시청률 평균을 다르게 계산하였습니다.
# 방송지속시간 동안의 이전셀에서 계산한 시청률 평균을 모두 더한 후, 방송 지속시간으로 평균을 낸 것이다.

In [51]:
X['월일'] = X['월'].astype(str) + X['일'].astype(str)
X.loc[:,'시분'] = X.loc[:,'시분'].astype(str)
X['시간열']=X['시분'].str.split(':').str[0].astype(str) + X['시분'].str.split(':').str[1].astype(str)
X['시간열']=X['시간열'].astype(int)

In [52]:
for i in range(len(X)):
    if X.loc[i,'시간열'] < 600:
         X.loc[i,'시간열']  += 2400

In [55]:
X['정수노출(분)']=X['노출(분)'].round(0).astype(int)
X['시청률평균'] = 0

In [56]:
for idx in range(len(X)-1):
    if X.loc[idx,'월일'] in time_list:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열'] :
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : 
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] : 
             X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
    else:
        if X.loc[idx,'시간열'] == X.loc[idx+1,'시간열']: 
            X.loc[idx,'시청률평균'] = 1
        elif X.loc[idx,'시간열'] > X.loc[idx+1,'시간열'] : 
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
        elif X.loc[idx,'시간열'] < X.loc[idx+1,'시간열'] :
             X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])

In [57]:
idx = len(X)-1
if X.loc[idx,'월일'] in time_list:
    X.loc[idx,'시청률평균'] = np.mean(weekend.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])
else:
    X.loc[idx,'시청률평균'] = np.mean(weekday.loc[X.loc[idx,'시간열']:(X.loc[idx,'시간열']+X.loc[idx,'정수노출(분)']),'시청률 평균'])

In [58]:
for idx in range(len(X)-1,-1,-1): 
    if X.loc[idx,'시청률평균'] == 1: 
        X.loc[idx,'시청률평균'] = X.loc[idx+1,'시청률평균'] 

## 상품군 결측치 처리

In [59]:
# 판매단가가 0 인 상품은 예측에서 제외

In [60]:
def NA_goods(df=X):
    df=df.loc[df['판매단가'] != 0,:]
    df = df.reset_index(drop=True)
    return df

In [61]:
X = NA_goods(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,날짜,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20,0.002632
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,2019-01-01,2019,01,01,6.000000,06:00,0101,600,20,0.002632
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20,0.002434
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,2019-01-01,2019,01,01,6.333333,06:20,0101,620,20,0.002434
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,2019-01-01,2019,01,01,6.666667,06:40,0101,640,20,0.003162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,2019-12-31,2019,12,31,23.666666,23:40,1231,2340,20,0.004741
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,2020-01-01,2020,01,01,0.000000,00:00,0101,2400,20,0.004069
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,2020-01-01,2020,01,01,0.000000,00:00,0101,2400,20,0.004069
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,2020-01-01,2020,01,01,0.000000,00:00,0101,2400,20,0.004069


##  토요일 결측치 처리

매주 토요일 18:00~18:20은 정보방송시간으로 추정 제외(당사사정에 따라 ±20 편성 변경 있음)

In [64]:
def NA_Saturday(df = X):
     # 년, 월, 일을 대입하면 요일을 반환하는 함수
    t = ['월','화','수','목','금','토','일']
    r = datetime.datetime.today()
    def printDayOfTheWeek(year, month, day):
        dayOfTheWeek = ['월','화','수','목','금','토','일']
        return dayOfTheWeek[datetime.datetime(year,month,day).weekday()]

    day_list = []
    # 방송일시 열에서 년, 월, 일, 요일, 시간을 뽑아내기
    for date in X.iloc[:,0]:
        date = str(date)
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])
        first = printDayOfTheWeek(year,month,day)
        second = date[11:16]
        # 각 행의 요일과 시간을 하나의 문자열로 만들어 day_list 리스트에 저장
        day_list.append(first+second)
    # train set에 요일&시간 열 추가
    X["요일/시간"] = day_list
    # 토요일 18시 0~19분에 시작하는 모든 방송 제거하기
    for i in range(len(df)):
        time = df.loc[i,'요일/시간']
        if (time[0:3]=="토18" and int(time[4:])<20):
            df = df.drop(i)
    return df

In [65]:
#checking
for i in range(len(NA_Saturday(X))):
    time = X.loc[i,'요일/시간'] # 여기 loc 로 바꾸었습니다. 왜냐하면 앞에서 평균방송분 열을 추가해야되서 열이 하나 늘어났어요
    if time[0:3]=="토18":
        print(time)

토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:30
토18:40
토18:50
토18:20
토18:50
토18:20
토18:30
토18:40
토18:50
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:40
토18:20
토18:20
토18:20
토18:20
토18:20
토18:20
토18:40
토18:40
토18:40
토18:40
토18:40
토18:40


In [66]:
X = NA_Saturday(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균,요일/시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,2019,01,01,6.666667,06:40,0101,640,20,0.003162,화06:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,2019,12,31,23.666666,23:40,1231,2340,20,0.004741,화23:40
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,2020,01,01,0.000000,00:00,0101,2400,20,0.004069,수00:00


힘들게 코드를 짰으나 토요일 18:00~18:20에 해당하는 방송은 없었다.............................

## 취급액 공백(이전 자료 50000)-> 제거

In [67]:
def NA_revenue(df=X):
    df.dropna(subset=['취급액'],inplace=True) 
    df=df.reset_index(drop=True)
    
    return df ## 주최측 요구에 따라 취급액이 NaN(공백)인 행 전체를 삭제

In [68]:
X = NA_revenue(X) ; X.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,년,월,일,시각,시분,월일,시간열,정수노출(분),시청률평균,요일/시간
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,2019,01,01,6.000000,06:00,0101,600,20,0.002632,화06:00
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,2019,01,01,6.333333,06:20,0101,620,20,0.002434,화06:20
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,2019,01,01,6.666667,06:40,0101,640,20,0.003162,화06:40


In [69]:
X.shape

(35379, 21)

## 시간 변수 세분화

특정 시간대에 홈쇼핑 방송 시청이 활발하여 홈쇼핑 상품 매출이 높을 것이라 생각하여 방송시간을 2시간으로 세분화한 시간 변수를 추가했다.

In [70]:
def time_2h(df = X):
    # 2시간으로 세분화한 시간 변수 열을 추가한다. 우선 0을 원소로 넣는다.
    df['06:00~08:00'] = 0
    df['08:00~10:00'] = 0
    df['10:00~12:00'] = 0
    df['12:00~14:00'] = 0
    df['14:00~16:00'] = 0
    df['16:00~18:00'] = 0
    df['18:00~20:00'] = 0
    df['20:00~22:00'] = 0
    df['22:00~00:00'] = 0
    df['00:00~02:00'] = 0
    df['02:00~04:00'] = 0
    
    # 대소 비교가 용이한 '시간열'변수를 활용해 세분화한 시간대에 따라 1을 채워 넣는다.
    df.loc[(600<=df['시간열'])&(df['시간열']<800),'06:00~08:00'] = 1
    df.loc[(800<=df['시간열'])&(df['시간열']<1000),'08:00~10:00'] = 1
    df.loc[(1000<=df['시간열'])&(df['시간열']<1200),'10:00~12:00'] = 1
    df.loc[(1200<=df['시간열'])&(df['시간열']<1400),'12:00~14:00'] = 1
    df.loc[(1400<=df['시간열'])&(df['시간열']<1600),'14:00~16:00'] = 1
    df.loc[(1600<=df['시간열'])&(df['시간열']<1800),'16:00~18:00'] = 1
    df.loc[(1800<=df['시간열'])&(df['시간열']<2000),'18:00~20:00'] = 1
    df.loc[(2000<=df['시간열'])&(df['시간열']<2200),'20:00~22:00'] = 1
    df.loc[(2200<=df['시간열'])&(df['시간열']<2400),'22:00~00:00'] = 1
    df.loc[(2400<=df['시간열'])&(df['시간열']<2600),'00:00~02:00'] = 1
    df.loc[(2600<=df['시간열'])&(df['시간열']<2800),'02:00~04:00'] = 1
    return(df)

In [71]:
X = time_2h(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,08:00~10:00,10:00~12:00,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,0,0,0,0,1,0,0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,0,0,0,0,0,0,1,0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,0,0,0,0,0,0,1,0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,0,0,0,0,0,0,1,0


## 요일별 세분화

요일에 따라 홈쇼핑 방송 시청이나 홈쇼핑 상품 매출에 유의미한 차이가 있을 것이라 생각하여 요일에 따라 세분화한 시간 변수를 추가했다.

In [72]:
def dayoftheweek(df = X):
    dayday = []
    # 기존의 '요일/시간' 변수에서 구한 요일만 따로 추출하여 범주형 변수를 만든다.
    for day in X.loc[:,'요일/시간']:
        onlyday = str(day[:1])
        dayday.append(onlyday)
            
    X["요일"] = dayday
    return df

In [73]:
X = dayoftheweek(X)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,10:00~12:00,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,요일
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,화
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,0,0,0,0,0,0,화
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,화
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,0,0,0,0,0,0,화
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,0,0,0,0,0,0,화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,0,0,0,1,0,0,화
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,0,0,0,0,0,1,0,수
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,0,0,0,0,0,1,0,수
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,0,0,0,0,0,1,0,수


## 분기별 세분화

분기별로 유의한 차이가 있을거라고 판단하고, 분기별로도 세분화를 진행하였다.

In [74]:
X['분기']=0 ## 분기별로 취급액에 유의미한 차이를 줄 수 있음을 시각화를 통해 확인

In [75]:
def quarter(df=X):
    for i in range(len(df)):
        
        if X.loc[i,'월'] in ['01','02','03']:
            X.loc[i,'분기']="1분기"
            
        elif X.loc[i,'월'] in ['04','05','06']:
            X.loc[i,'분기']="2분기"
            
        elif X.loc[i,'월'] in ['07','08','09']:
            X.loc[i,'분기']="3분기"
            
        else:
            X.loc[i,'분기']="4분기"
            
    return df ##1,2,3 / 4,5,6 / 7,8,9 / 10,11,12월 로 1~4분기 구분하여 categorical로 변환

In [76]:
quarter(X);X.head() 

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,12:00~14:00,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,0,0,0,0,0,화,1분기
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,0,0,0,0,0,화,1분기
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,0,0,0,0,0,화,1분기
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,0,0,0,0,0,화,1분기
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,0,0,0,0,0,화,1분기


## 강수량 변수

강수량 데이터의 경우 기상청의 기상자료개방포털의 공공데이터를 사용했다.(https://data.kma.go.kr/stcs/grnd/grndRnList.do?pgmNo=69)
강수량이 높으면 집에서 활동하는 인구가 늘어 홈쇼핑 매출에 큰 영향이 있을 것이라 생각했고, 이에 일별 강수량을 고려했다.

In [77]:
# 외부데이터 불러오기
# 이 경우에도 외부데이터의 이름은 19rain.csv 의 이름으로 저장하여 다.
rain_df=pd.read_csv(directory+'/19rain.csv',header = 7, encoding='CP949')
rain = rain_df.copy()

In [78]:
rain.head()

,날짜,지점,강수량(mm)
0,2019-01-01,전국,0.1
1,2019-01-02,전국,0.0
2,2019-01-03,전국,0.0
3,2019-01-04,전국,0.0
4,2019-01-05,전국,0.0


In [79]:
# 날짜에 맞게 강수량 데이터를 병합 
def rain_feature(df = X):
       
    df = pd.merge(df, rain, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)

X = rain_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,14:00~16:00,16:00~18:00,18:00~20:00,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기,강수량(mm)
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,0,0,0,0,화,1분기,0.1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,0,0,0,0,화,1분기,0.1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,0,0,0,0,화,1분기,0.1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,0,0,0,0,화,1분기,0.1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,0,0,0,0,화,1분기,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,0,1,0,0,화,4분기,0.1
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,0,0,0,1,0,수,1분기,0.0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,0,0,0,1,0,수,1분기,0.0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,0,0,0,1,0,수,1분기,0.0


## 온도 변수

기온 데이터 또한 마찬가지로 기상청의 기상자료개방포털의 공공데이터를 사용했다.(https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70)
기온에 따라 특정 상품군의 매출이 달라질 것이라 생각하여 일별 기온(평균기온, 최고기온, 최저기온)을 고려했다.

In [80]:
# 외부데이터 불러오기
# 기온 자료 또한 19temp.csv 의 이름으로 저장한 뒤 불러온다.
temp_df=pd.read_csv(directory+'/19temp.csv',header = 7, encoding='CP949')
temp = temp_df.copy()

In [81]:
temp.head()

,날짜,지점,평균기온(℃),최저기온(℃),최고기온(℃)
0,2019-01-01,전국,-2.1,-5.8,2.1
1,2019-01-02,전국,-2.5,-7.0,3.2
2,2019-01-03,전국,-2.1,-7.7,5.0
3,2019-01-04,전국,-0.7,-7.3,5.3
4,2019-01-05,전국,0.2,-4.6,5.3


In [82]:
# 날짜에 맞게 기온 데이터를 병합
def temp_feature(df = X):
    df = pd.merge(df, temp, on="날짜", how= "inner")
    df.drop(['지점'],axis=1,inplace = True)
    return(df)
X = temp_feature(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,20:00~22:00,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃)
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,1,0,0,화,4분기,0.1,-3.8,-6.7,0.4
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,1,0,수,1분기,0.0,4.1,-6.5,4.1
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,1,0,수,1분기,0.0,4.1,-6.5,4.1
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,1,0,수,1분기,0.0,4.1,-6.5,4.1


## 연예인 변수 

상품명에 연예인 이름이 포함되는 경우 상품에 대한 평판과 신뢰도가 상승하여 매출에 긍정적인 효과가 있을 것으로 판단하였다. 연예인 변수를 이진변수로 추가하였다. 

In [83]:
#연예인 이름이 포함되는 상품은 "연예인" 변수에서 1의 값을, 포함되지 않는 상품은 0의 값을 가진다. 
def celebrity(df=X):
    df['연예인']=0
    list_celebrity=["전지현", "팽현숙", "전철우", "강레오", "김선영", "김정배",
                    "김규흔", "이봉원", "오세득", "유귀열", "이경제", "이만기",
                    "이보은", "임성근", "전철우", "최인선", "김병만", "김병지", 
                    "이동수", "서장훈", "송도순", "효재", "천수봉", 
                    "김정문", "숀리", "이정섭", "황성주"]
    for i in list_celebrity:
        df.loc[df.상품명.str.contains(i),'연예인']=1
    return df

In [84]:
X=celebrity(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,22:00~00:00,00:00~02:00,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,1,0,0,화,4분기,0.1,-3.8,-6.7,0.4,0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,1,0,수,1분기,0.0,4.1,-6.5,4.1,0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,1,0,수,1분기,0.0,4.1,-6.5,4.1,0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,1,0,수,1분기,0.0,4.1,-6.5,4.1,0


In [85]:
print(X.sum().연예인) #연예인 이름을 사용한 총 데이터 수

923


## 월급날

월급날에는 홈쇼핑 고객이 소비지출을 더 많이 하여 홈쇼핑 매출에도 긍정적인 영향을 줄 것으로 생각했으며 월급수령으로 인한 매출의 효과를 고려하기 위해 "월급날" 변수를 추가하였다. 고객 별로 월급 지급날이 상이하기 때문에 국민은행 공식 블로그를 참고하여 월급날을 10일, 20일, 25일로 정했다. 대체로 10일은 중소기업, 20일은 일부 공무원, 25일은 대기업의 월급 날이다. 월급 수령 후 소비지출의 증가 효과는 수령 당일 이후 며칠동안 더 지속될 것으로 생각하여 각 월급날의 +3일까지 "월급날" 변수에 포함했다. 

In [86]:
#https://news.sbs.co.kr/news/endPage.do?news_id=N1005505452
#https://m.blog.naver.com/youngkbblog/221714986107

In [87]:
#10일~13일, 20~23일, 25일~28일에 판매한 상품은 "월급날" 변수에서 1의 값을, 그 이외 날은 0의 값을 갖는다. 
def income(df = X):
    df['월급날'] = 0
    df.loc[ ((10<=df['일'].astype('float32'))&(df['일'].astype('float32')<=13)) | ((20<=df['일'].astype('float32'))&(df['일'].astype('float32')<= 23)) | ((25<=df['일'].astype('float32')) & (df['일'].astype('float32')<= 28)),'월급날']  = 1
    return(df)

In [88]:
X=income(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,00:00~02:00,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,화,4분기,0.1,-3.8,-6.7,0.4,0,0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,1,0,수,1분기,0.0,4.1,-6.5,4.1,0,0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,1,0,수,1분기,0.0,4.1,-6.5,4.1,0,0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,1,0,수,1분기,0.0,4.1,-6.5,4.1,0,0


In [89]:
X.loc[X["일"]=="10", ["월급날"]] #잘 들어갔는지 확인
print(X.sum().월급날) #총 월급날

13650


## 금,토 및 공휴일

특히 직장인 같은 경우 금요일 저녁부터 토요일 및 공휴일에 집에서 보내는 시간이 많아져 홈쇼핑을 더 많이 접할 것으로 판단하였다. 이에 금요일, 토요일, 및 공휴일이 홈쇼핑 매출에 긍정적인 영향을 줄 것으로 생각했다. 

In [90]:
#https://0muwon.com/entry/2019%EB%85%84-%ED%9C%B4%EC%9D%BC-%EA%B3%B5%ED%9C%B4%EC%9D%BC-%EB%8C%80%EC%B2%B4%ED%9C%B4%EC%9D%BC-%ED%99%A9%EA%B8%88%EC%97%B0%ED%9C%B4-%EC%9B%94%EB%B3%84-%EC%99%84%EB%B2%BD%EC%A0%95%EB%A6%AC

In [91]:
#2019년 각 월에 대해 금요일, 토요일 및 공휴일 날짜 리스트를 뽑았다.
#해당 날짜에 편성된 상품은 '휴일' 변수에서 1의 값을, 다른 날에 편성된 상품은 0의 값을 부여했다. 
def holiday(df = X):
    df["휴일"] = 0
    holiday_list=[[1,4,5,6,11,12,13,18,19,20,25,26,27],
                  [1,2,3,4,5,6,8,9,10,15,16,17,22,23,24],
                  [1,2,3,8,9,10,15,16,17,22,23,24,29,30,31],
                  [5,6,7,12,13,14,19,20,21,26,27,28],
                  [3,4,5,6,10,11,12,17,18,19,24,25,26,31],
                  [1,2,6,7,8,9,14,15,16,21,22,23,28,29,30],
                  [5,6,7,12,13,14,19,20,21,26,27,28],
                  [2,3,4,9,10,11,15,16,17,18,23,24,25,30,31],
                  [1,6,7,8,12,13,14,15,20,21,22,27,28,29],
                  [3,4,5,6,9,11,12,13,18,19,20,25,26,27],
                  [1,2,3,8,9,10,15,16,17,22,23,24,29,30],
                  [1,6,7,8,13,14,15,20,21,22,25,27,28,29]]
    for i, holiday in enumerate(holiday_list):
        for j in holiday:
            df.loc[ (df["월"].astype("float32")==i+1) & (df["일"].astype("float32")==j),'휴일'] = 1
    return(df)

In [92]:
X=holiday(X);X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,02:00~04:00,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,화,4분기,0.1,-3.8,-6.7,0.4,0,0,0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,수,1분기,0.0,4.1,-6.5,4.1,0,0,1
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,수,1분기,0.0,4.1,-6.5,4.1,0,0,1
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,수,1분기,0.0,4.1,-6.5,4.1,0,0,1


In [93]:
X.loc[(X["월"].astype("float32")==3) & (X["일"].astype("float32")==1),["휴일"]] #잘 들어갔는지 확인
print(X.sum().휴일) #총 휴일

16136


## 상품군 power

각 상품군 별로 '(취급액/판매단가)'의 총합과 평균방송분의 총 합을 구한다 그리고  각 상품군 별로 '(취급액/판매단가)의 총 합'을 '평균방송분'의 총 합으로 나누고 소수점아래 둘째 자리에서 반올림해 분당 판매개수를 구하고 그것을 상품군의 power(얼마나 팔리는지) 라고 정의하였다.

In [94]:
def goods_power(df = X):
    df["상품군power"] = 0
    # 각 상품군 별로 '(취급액/판매단가)'의 총합과 평균방송분의 총 합을 구한다
    for name in ('가구','가전','건강기능','농수축','생활용품','속옷','의류','이미용','잡화','주방','침구'):
        revenue = 0
        runtime = 0
        for i in range(len(df)):
            if df.loc[i,'상품군'] == name:
                revenue += df.loc[i,'취급액']/df.loc[i,'판매단가']
                runtime += df.loc[i,'평균방송분']
            else:
                pass
        # 각 상품군 별로 '(취급액/판매단가)의 총 합'을 '평균방송분'의 총 합으로 나누고 소수점아래 둘째 자리에서 반올림해 분당 판매개수를 구한다
        for i in range(len(df)):
            if df.loc[i,'상품군'] == name:
                df.loc[i,'상품군power'] = round(revenue/runtime,2)
    
    return(df)

In [95]:
X = goods_power(X) ; X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,요일,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,화,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,화,4분기,0.1,-3.8,-6.7,0.4,0,0,0,19.40
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,수,1분기,0.0,4.1,-6.5,4.1,0,0,1,19.40
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,수,1분기,0.0,4.1,-6.5,4.1,0,0,1,19.40
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,수,1분기,0.0,4.1,-6.5,4.1,0,0,1,19.40


## 소비자물가지수

In [96]:
# 통계청의 소비자물가지수를 참고해 월별로 해당 수치를 추가해준다.
# 출처: http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1060
def CPI(df = X):
    df['CPI'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    cpi1,cpi2,cpi3,cpi4,cpi5,cpi6,cpi7,cpi8,cpi9,cpi10,cpi11,cpi12,cpi13 = 0.8, 0.5, 0.4, 0.6, 0.7, 0.7, 0.6, 0.0, -0.4, 0.0, 0.2, 0.7, 1.5
    df.loc[df['연월']== '1901','CPI'] = cpi1
    df.loc[df['연월']== '1902','CPI'] = cpi2
    df.loc[df['연월']== '1903','CPI'] = cpi3
    df.loc[df['연월']== '1904','CPI'] = cpi4
    df.loc[df['연월']== '1905','CPI'] = cpi5
    df.loc[df['연월']== '1906','CPI'] = cpi6
    df.loc[df['연월']== '1907','CPI'] = cpi7
    df.loc[df['연월']== '1908','CPI'] = cpi8
    df.loc[df['연월']== '1909','CPI'] = cpi9
    df.loc[df['연월']== '1910','CPI'] = cpi10
    df.loc[df['연월']== '1911','CPI'] = cpi11
    df.loc[df['연월']== '1912','CPI'] = cpi12
    df.loc[df['연월']== '2001','CPI'] = cpi13
    del df['연월']
    
    return(df)

In [97]:
X = CPI(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,분기,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,1분기,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,4분기,0.1,-3.8,-6.7,0.4,0,0,0,19.40,0.7
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,1분기,0.0,4.1,-6.5,4.1,0,0,1,19.40,1.5
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,1분기,0.0,4.1,-6.5,4.1,0,0,1,19.40,1.5
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,1분기,0.0,4.1,-6.5,4.1,0,0,1,19.40,1.5


## 소비자 동향지수

In [98]:
# 통계청의 소비자동향지수를 참고해 월별로 해당 수치를 추가해준다.
# 출처: http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=1058
def CSI(df = X):
    df['CSI'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    csi1,csi2,csi3,csi4,csi5,csi6,csi7,csi8,csi9,csi10,csi11,csi12,csi13 = 97.5, 99.5, 99.8, 101.6, 97.9, 97.5, 95.9, 92.5, 96.9, 98.6, 101.0, 100.5, 104.2
    df.loc[df['연월']== '1901','CSI'] = csi1
    df.loc[df['연월']== '1902','CSI'] = csi2
    df.loc[df['연월']== '1903','CSI'] = csi3
    df.loc[df['연월']== '1904','CSI'] = csi4
    df.loc[df['연월']== '1905','CSI'] = csi5
    df.loc[df['연월']== '1906','CSI'] = csi6
    df.loc[df['연월']== '1907','CSI'] = csi7
    df.loc[df['연월']== '1908','CSI'] = csi8
    df.loc[df['연월']== '1909','CSI'] = csi9
    df.loc[df['연월']== '1910','CSI'] = csi10
    df.loc[df['연월']== '1911','CSI'] = csi11
    df.loc[df['연월']== '1912','CSI'] = csi12
    df.loc[df['연월']== '2001','CSI'] = csi13
    del df['연월']
    
    return(df)

In [99]:
X = CSI(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,강수량(mm),평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0.1,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0.1,-3.8,-6.7,0.4,0,0,0,19.40,0.7,100.5
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0.0,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0.0,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0.0,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2


## 소매업태별(홈쇼핑) 판매액지수(불변지수)

In [100]:
# 통계청의 소매업태별 판매액지수 중 홈쇼핑 항목의 불변지수를 참고해 월별로 해당 수치를 추가해준다.
# 출처: http://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1K31013&vw_cd=MT_ZTITLE&list_id=JG2&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE
def RS(df = X):
    df['RS'] = 0
    day_list = []
    for date in df.loc[:,'방송일시']:
        year = date[2:4]
        month = date[5:7]
        day_list.append(year+month)
    df['연월'] = day_list
    x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13 = 139.3, 113.3, 134.0, 130.1, 133.8, 123.4, 124.7, 120.3, 124.3, 139.0, 138.8, 138.0, 131.7
    df.loc[df['연월']== '1901','RS'] = x1
    df.loc[df['연월']== '1902','RS'] = x2
    df.loc[df['연월']== '1903','RS'] = x3
    df.loc[df['연월']== '1904','RS'] = x4
    df.loc[df['연월']== '1905','RS'] = x5
    df.loc[df['연월']== '1906','RS'] = x6
    df.loc[df['연월']== '1907','RS'] = x7
    df.loc[df['연월']== '1908','RS'] = x8
    df.loc[df['연월']== '1909','RS'] = x9
    df.loc[df['연월']== '1910','RS'] = x10
    df.loc[df['연월']== '1911','RS'] = x11
    df.loc[df['연월']== '1912','RS'] = x12
    df.loc[df['연월']== '2001','RS'] = x13
    del df['연월']
    
    return(df)

In [101]:
X = RS(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI,RS
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,-3.8,-6.7,0.4,0,0,0,19.40,0.7,100.5,138.0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2,131.7
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2,131.7
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2,131.7


## 심야 시간대 표현

앞의 시각 변수에서 00 시 이후는 24를 더해주어서, 23:30 시와 00:30 시 의 경우 늦은 밤으로 매우 비슷하지만 값이 크게 차이가 나는 경우를 보정해주었다.

In [102]:
X.loc[X.loc[:,'시각'] < 3,'시각'] = X.loc[X.loc[:,'시각'] < 3,'시각'] + 24

In [103]:
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,평균기온(℃),최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI,RS
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,-2.1,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,-3.8,-6.7,0.4,0,0,0,19.40,0.7,100.5,138.0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2,131.7
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2,131.7
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,4.1,-6.5,4.1,0,0,1,19.40,1.5,104.2,131.7


## Prime Time

In [104]:
# https://www.etnews.com/20180718000049?m=1
# 자료 조사결과, 프라임타임이 홈쇼핑 매출액에 중요한 변수가 될 수 있음을 확인
# 특히 ns shop +의 경우 식품(농수축)을 주요 타겟으로 하기에 식품 프라임타임(16~18시)을 별도로 취급.
# 기타 상품군도 고려하여 오전/오후에 각각 프라임타임 지정.

In [105]:
def prime_time(df = X):
    df['PrimeTime'] = 0
    
    for i in range(len(df)):
        if (9 <= df.loc[i,'시각']) and (df.loc[i,'시각'] < 11):
            df.loc[i,'PrimeTime'] = "오전프라임"
        elif df.loc[i,'상품군']=="농수축" and (16<=df.loc[i,'시각']) and (df.loc[i,'시각']< 18):
            df.loc[i,'PrimeTime'] = "식품프라임"            
        elif df.loc[i,'상품군']!="농수축" and (20<=df.loc[i,'시각']) and (df.loc[i,'시각']< 23):
            df.loc[i,'PrimeTime'] = "그외프라임"
        else:
            df.loc[i,'PrimeTime'] = "프라임아님"
    
    return df

## 방송 시작시간을 고려해서 [,)로 시간 처리
## 프라임타임을 세분화해서 오전/식품오후/그외오후/프라임아님 4가지 카테고리로 구분

In [106]:
prime_time(X) ; X.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,최저기온(℃),최고기온(℃),연예인,월급날,휴일,상품군power,CPI,CSI,RS,PrimeTime
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3,프라임아님
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3,프라임아님
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3,프라임아님
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3,프라임아님
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,-5.8,2.1,0,0,1,26.41,0.8,97.5,139.3,프라임아님


## 남성, 여성 상품

의류, 속옷, 잡화, 이미용 상품 중 남녀 별로 구분할 수 있는 상품이 많다. 2017년 KISDI에서 실시한 연구에 따르면 TV홈쇼핑 이용자 중 약 70%가 여성이다. 이렇게 남녀별로 홈쇼핑 소비패턴이 다르기 때문에 성별을 고려하였다. 

In [107]:
#성별을 고려하기 위해 각 성별을 더미변수로 만들었다. 
#의류, 속옷, 잡화, 이미용 상품 중에서 남성 상품은 "남성" 변수에서 1의 값을, 여성 상품은 "여성" 변수에서 1의 값을 부여했다. 
#남녀 구분이 명확하지 않은 상품은 "남성"과 "여성" 변수에서 0의 값을 갖는다. 
def gender(df=X):
    df["남성상품"]=0
    df["여성상품"]=0
    list_male=["남성","드로즈","푸마 드라이셀라인 뉴 런닝","언더셔츠","카파 런닝","리복 스피드윅 런닝",
               "트렁크","프로스펙스 2020 퍼포먼스 런닝"]
    list_female=["여성","브라","밍크","팬티","치마","레깅스","란쥬","핸드백",
                 "앙상블","엔셀라두스 밴딩팬츠","레이스","엘리자베스아덴","버켓 백","미란다백","모터백",
                 "크로스바디 백","크로스바디백","립스틱","쿠션","젤네일","아이라이너","사첼백","숄더","속눈썹","토트백","호보백",
                 "원피스","틴트","투웨이백","이지탑","크로스백","클러치백","보정런닝", "아이스타일러",
                 "오렐리안 히트모 스킨컬렉션 기모런닝","보정 런닝", "노벨라백", "제니스시계 주얼리세트",
                 "콘체르토 다이아몬드 워치(주얼리3종)", "삭루츠 버켓 백",
                 "무빙스타 다이아몬드 워치", "목걸이", "팔찌", "18K 베르니아 주얼리 세트",
                 "귀걸이", "반지", "펌프스", "링", "사첼", "실크테라피 오리지널 에센스 패키지", "마스카라",
                 "피부약방 더오리진 크림세트"]
    
    for i in ["의류","속옷","잡화","이미용"]:
        for j in list_male:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(j)),"남성상품"]=1
        for k in list_female:
            df.loc[(df["상품군"]==i)&(df.상품명.str.contains(k)),"여성상품"]=1
    return df

In [108]:
X = gender(X); X

C:\Users\goran\Anaconda3\envs\stat\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,연예인,월급날,휴일,상품군power,CPI,CSI,RS,PrimeTime,남성상품,여성상품
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0,0,1,26.41,0.8,97.5,139.3,프라임아님,1,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0,0,1,26.41,0.8,97.5,139.3,프라임아님,0,1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0,0,1,26.41,0.8,97.5,139.3,프라임아님,1,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0,0,1,26.41,0.8,97.5,139.3,프라임아님,0,1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0,0,1,26.41,0.8,97.5,139.3,프라임아님,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,0,0,19.40,0.7,100.5,138.0,프라임아님,0,0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,0,0,1,19.40,1.5,104.2,131.7,프라임아님,0,0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,0,0,1,19.40,1.5,104.2,131.7,프라임아님,0,0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,0,0,1,19.40,1.5,104.2,131.7,프라임아님,0,0


## 무이자, 일시불 처리

무이자, 일시불과 같은 구매 방식 또한 소비에 영향을 미칠 것이라 판단하여 상품 별 무이자, 일시불을 나타내는 열을 추가했다.

In [109]:
def pay(df=X):
    df["무이자"]=0 
    df["일시불"]=0
    pay_in_inst=["무)", "무이자"] # 무), (무), 무이자 등의 형태로 무이자 상품이 표기
    pay_in_full=["일)", "일시불"] # 일), (일), 일시불 등의 형태로 무이자 상품이 표기
    for i in pay_in_inst:
        df.loc[(df.상품명.str.contains(i,regex=False))&(~df.상품명.str.contains("보험")),"무이자"]=1 # (무)로 표기된 무배당 보험상품 제외
    for j in pay_in_full:
        df.loc[(df.상품명.str.contains(j, regex=False))&(~df.상품명.str.contains("세일")),"일시불"]=1 # (세일)로 표기된 세일 상품 제외
    return df

In [110]:
X = pay(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,휴일,상품군power,CPI,CSI,RS,PrimeTime,남성상품,여성상품,무이자,일시불
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,1,26.41,0.8,97.5,139.3,프라임아님,1,0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,1,26.41,0.8,97.5,139.3,프라임아님,0,1,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,1,26.41,0.8,97.5,139.3,프라임아님,1,0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,1,26.41,0.8,97.5,139.3,프라임아님,0,1,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,1,26.41,0.8,97.5,139.3,프라임아님,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0,19.40,0.7,100.5,138.0,프라임아님,0,0,0,1
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,1,19.40,1.5,104.2,131.7,프라임아님,0,0,1,0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,1,19.40,1.5,104.2,131.7,프라임아님,0,0,0,1
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,1,19.40,1.5,104.2,131.7,프라임아님,0,0,1,0


## 유명 기업/브랜드명

In [112]:
# 유명기업/브랜드 명 포함된 상품에 대해서 1 의 값을 가지는 원핫 벡터 열을 추가하였다.
def brandname(df=X):
    df['유명기업/브랜드']=0
    lst=["LG","SPC삼립","경남제약", "광동", "구찌", "굿프렌드", "농협", "딤채", "디즈니",
         "락앤락","리복","마리끌레르","뱅뱅","버버리", "베지밀","보루네오","본죽","삼성",
         "엘르","종근당건강","쿠쿠",'쿠첸',"코치","푸마","프라다","하림","한샘","버버리",
         "비비고"]
    for i in lst:
        df.loc[df.상품명.str.contains(i),'유명기업/브랜드']=1
    for i in range(len(df)):
        if ("폴로" in df.loc[i,"상품명"]) or ("USPA" in df.loc[i,"상품명"]):
            df.loc[i,"유명기업/브랜드"] = 1
            
    return df

In [113]:
X = brandname(X); X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,상품군power,CPI,CSI,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,26.41,0.8,97.5,139.3,프라임아님,1,0,0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,26.41,0.8,97.5,139.3,프라임아님,0,1,0,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,26.41,0.8,97.5,139.3,프라임아님,1,0,0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,26.41,0.8,97.5,139.3,프라임아님,0,1,0,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,26.41,0.8,97.5,139.3,프라임아님,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,19.40,0.7,100.5,138.0,프라임아님,0,0,0,1,1
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,19.40,1.5,104.2,131.7,프라임아님,0,0,1,0,1
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,19.40,1.5,104.2,131.7,프라임아님,0,0,0,1,1
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,19.40,1.5,104.2,131.7,프라임아님,0,0,1,0,1


## 타 채널 시청자 수

논문 '홈쇼핑 주문 건수의 확산과 zapping의 효과 분석' 등의 자료를 통해 다른 방송을 시청하다가 중간에 홈쇼핑 채널로 유입되는 경우가 홈쇼핑 방송 시청에 유의미한 영향을 미친다고 생각했다. 따라서 '닐슨코리아'의 2019년 전국 시청자수 상위 20위 방송의 시청자수 데이터를 가공하여 이를 반영하려 했다. 분석 자료에 날짜별 전국 시청자수 상위 20위 방송의 시청자수의 평균을 변수로 추가했다.

In [116]:
# 크롤링에 필요한 패키지 import
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [117]:
# 웹 주소에 들어갈 날짜 부분을 리스트로 정의
date = X['월일']
date = date[~date.duplicated()]
date = list(date)
date1 = date[:100]
date2 = date[100:200]
date3 = date[200:300]
date4 = date[300:]

In [118]:
X['타 채널 시청자 수 평균'] = 0
def rate_add(df=X, date=date1):
    # 앞서 설정한 날짜 리스트를 통해 날짜 별 시청률 웹 페이지 호출
    for i in date:
        url = "http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date=2019"+i
        html = urlopen(url)  
        rate_html = BeautifulSoup(html, "html.parser") 
        rate = []
        # html 형식의 값을 다듬는다
        for j in range(10,20):
            a = rate_html.find_all("td",class_='percent')[j].get_text()
            a=a.replace("\t", "")
            a=a.replace("\r","")
            a=a.replace("\n","")
            a=a.replace(",","")
            a=a.strip()
            rate.append(a)
            
        for k in range(10,20):
            b = rate_html.find_all("td",class_='percent_g')[k].get_text()
            b=b.replace("\t", "")
            b=b.replace("\r","")
            b=b.replace("\n","")
            b=b.replace(",","")
            b=b.strip()
            rate.append(b)
        # 정수형으로 변환한 후 평균값을 도출   
        rate = [int(m) for m in rate]
        rate_mean = sum(rate) / 20
        df.loc[df['월일']==i,'타 채널 시청자 수 평균'] = rate_mean
           
    return  df

In [119]:
# 2020년 1월 1일의 데이터는 따로 추가
url = "http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date=20200101"
html = urlopen(url)  
rate_html = BeautifulSoup(html, "html.parser") 
rate = []
for j in range(10,20):
    a = rate_html.find_all("td",class_='percent')[j].get_text()
    a=a.replace("\t", "")
    a=a.replace("\r","")
    a=a.replace("\n","")
    a=a.replace(",","")
    a=a.strip()
    rate.append(a)
            
for k in range(10,20):
    b = rate_html.find_all("td",class_='percent_g')[k].get_text()
    b=b.replace("\t", "")
    b=b.replace("\r","")
    b=b.replace("\n","")
    b=b.replace(",","")
    b=b.strip()
    rate.append(b)
         
rate = [int(m) for m in rate]
rate_mean = sum(rate) / 20
X.loc[X['년']=='2020','타 채널 시청자 수 평균'] = rate_mean

In [120]:
X = rate_add(X, date1)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,CPI,CSI,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,0.8,97.5,139.3,프라임아님,1,0,0,0,0,1520.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,0.8,97.5,139.3,프라임아님,0,1,0,0,0,1520.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,0.8,97.5,139.3,프라임아님,1,0,0,0,0,1520.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,0.8,97.5,139.3,프라임아님,0,1,0,0,0,1520.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,0.8,97.5,139.3,프라임아님,1,0,0,0,0,1520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,0.7,100.5,138.0,프라임아님,0,0,0,1,1,0.0
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,1.5,104.2,131.7,프라임아님,0,0,1,0,1,1520.0
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,1.5,104.2,131.7,프라임아님,0,0,0,1,1,1520.0
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,1.5,104.2,131.7,프라임아님,0,0,1,0,1,1520.0


In [121]:
X = rate_add(X, date2)

In [122]:
X = rate_add(X, date3)

In [123]:
X = rate_add(X, date4)

## 가전제품 소분류

가전제품 중 세탁기와 냉장고의 취급액이 에어컨과 노트북의 취급액 보다 두 배 이상 차이났다. 가전제품의 종류에 따라 취급액 차이를 고려하기 위해 "가전제품 소분류" 변수를 생성했다. 

In [124]:
#가전제품 소분류는 총 10가지이다. 공기청정기를 "공청기"로 줄여서 명시한 상품도 있었기에 해당 상품도 공기청정기로 처리했다. 
X["가전제품"]="가전제품 아님"
list_products=['세탁기', '노트북', '냉장고', 'TV', '건조기', '공기청정기', '무선청소기', '에어컨', '의류관리기']

for j in list_products:
    for i in range(len(X)):
        if X.loc[i,"상품군"]=="가전":
            if j in X.loc[i,"상품명"]:
                X.loc[i,"가전제품"]=j

for i in range(len(X)):
    if X.loc[i,"상품군"]=="가전":
        if "공청기" in X.loc[i,"상품명"]:
            X.loc[i,"가전제품"]="공기청정기"

X.loc[47:150,['상품명',"가전제품"]]

,상품명,가전제품
47,무이자 LG 통돌이 세탁기,세탁기
48,일시불 LG 통돌이 세탁기,세탁기
49,무이자 LG 통돌이 세탁기,세탁기
50,마리노블 밍크 롱코트,가전제품 아님
51,마리노블 밍크 롱코트,가전제품 아님
...,...,...
146,일시불 LG 울트라HD TV 55UK6800HNC,TV
147,무이자 LG 울트라HD TV 55UK6800HNC,TV
148,일시불 LG 울트라HD TV 65UK6800HNC,TV
149,무이자 LG 울트라HD TV 65UK6800HNC,TV


## 농수축소분류

In [125]:
# 농수축 상품을 10개의 주요 품목으로 세분화
def AgroFishery(df=X):
    df['농수축소분류']="분류에없음"
    lst_쌀 = ["멋진밥상 흥양농협 햅쌀 20kg", "멋진밥상 흥양농협 쌀 20kg", "19년 멋진밥상 흥양농협 햅쌀 20kg"]
    lst_과일 = ["한라봉", "천혜향", "오렌지", "감말랭이", "사과", "샤인머스켓"]
    lst_김치 = ["김치"]
    lst_당류 = ["농협안심벌꿀 4병", "[농협안심벌꿀] 아카시아꿀 2병 + 야생화꿀 2병"]
    lst_어류 = ["광어", "옥돔", "굴비", "가자미", "병어", "오징어", "꼬막", "새우", "홍어", "골뱅이" ,"참조기", "동태포", 
                "해물", "재첩", "꽃게", "문어", "장어", "고등어", "전복", "랍스터",  "수산물", "낙지", "과메기",
                "피시원 국내산 손질 대구 8팩 * 매운탕 양념 8팩", "황태", "갈치", "김정배 명인 젓갈세트", "참굴"]
    lst_건어류 = ["쥐포", "더 그릴 3종 아구채4팩+양태채2팩+쥐포채2팩 + 소스8팩"]
    lst_육류 = ["곱창", "갈비", "장조림", "곰탕", "강레오의 탕요일", "오리로스", "돼지왕구이", "삼계탕", "치마살", "한우", 
                "소한마리", "우삽겹", "안창살", "도가니", "감자탕", "치킨스테이크", "닭발"]
    lst_음료 = ["매실원액", "두유", "배즙", "머루원액"]
    lst_장류조미료 = ["메주", "고춧가루", "다시팩"]
    lst_주전부리 = ["옥수수", "콩국수", "호빵", "고구마", "구워만든 곡물그대로", "영양 강정바",
                    "티각태각 부각", "모시떡", "식탁앤김", "맛밤", "선식", "곤드레나물밥"]
    lst_채소 = ["시래기", "매생이"]
    
    for i in lst_쌀:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="쌀"
    for i in lst_과일:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="과일"
    for i in lst_김치:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="김치"
    for i in lst_당류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="당류"
    for i in lst_어류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="어류"
    for i in lst_건어류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="건어류"
    for i in lst_육류:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="육류"
    for i in lst_음료:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="음료"
    for i in lst_장류조미료:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="장류/조미료"
    for i in lst_주전부리:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="주전부리"
    for i in lst_채소:
        df.loc[df.상품명.str.contains(i),'농수축소분류']="채소"


    return df

In [126]:
X = AgroFishery(X)
X

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,평균방송분,겹치는수,...,RS,PrimeTime,남성상품,여성상품,무이자,일시불,유명기업/브랜드,타 채널 시청자 수 평균,가전제품,농수축소분류
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,10.0,1,...,139.3,프라임아님,1,0,0,0,0,1520.0,가전제품 아님,분류에없음
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,10.0,1,...,139.3,프라임아님,0,1,0,0,0,1520.0,가전제품 아님,분류에없음
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,10.0,2,...,139.3,프라임아님,1,0,0,0,0,1520.0,가전제품 아님,분류에없음
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,10.0,2,...,139.3,프라임아님,0,1,0,0,0,1520.0,가전제품 아님,분류에없음
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,10.0,3,...,139.3,프라임아님,1,0,0,0,0,1520.0,가전제품 아님,분류에없음
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,5.0,2,...,138.0,프라임아님,0,0,0,1,1,1335.9,가전제품 아님,분류에없음
35375,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,5.0,3,...,131.7,프라임아님,0,0,1,0,1,1520.0,가전제품 아님,분류에없음
35376,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,5.0,3,...,131.7,프라임아님,0,0,0,1,1,1520.0,가전제품 아님,분류에없음
35377,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,5.0,3,...,131.7,프라임아님,0,0,1,0,1,1520.0,가전제품 아님,분류에없음


## 저장

In [129]:
X.to_excel('X0823.xlsx')

In [130]:
weekday.to_csv('weekday.csv')
weekend.to_csv('weekend.csv')